In [ ]:
from tqdm.notebook import tqdm
#import network
#import utils
import os
import random
import argparse
import numpy as np

from torch.utils import data
from time import perf_counter
#from datasets import VOCSegmentation, Cityscapes
#from utils import ext_transforms as et
#from metrics import StreamSegMetricsd
import time
import torch
import torch.nn as nn
#from utils.visualizer import Visualizer
from threading import Thread
import IPython

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

import pickle
import zipfile

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import collections

class ConvLayer(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel=3, stride=1, dropout=0.1):
        super().__init__()
        self.add_module('conv', nn.Conv2d(in_channels, out_channels, kernel_size=kernel,
                                          stride=stride, padding=kernel//2, bias = False))
        self.add_module('norm', nn.BatchNorm2d(out_channels))
        self.add_module('relu', nn.ReLU(inplace=True))

        #print(kernel, 'x', kernel, 'x', in_channels, 'x', out_channels)

    def forward(self, x):
        return super().forward(x)
        

class BRLayer(nn.Sequential):
    def __init__(self, in_channels):
        super().__init__()
        
        self.add_module('norm', nn.BatchNorm2d(in_channels))
        self.add_module('relu', nn.ReLU(True))
    def forward(self, x):
        return super().forward(x)


class HarDBlock_v2(nn.Module):
    def get_link(self, layer, base_ch, growth_rate, grmul):
        if layer == 0:
          return base_ch, 0, []
        out_channels = growth_rate
        link = []
        for i in range(10):
          dv = 2 ** i
          if layer % dv == 0:
            k = layer - dv
            link.insert(0, k)
            if i > 0:
                out_channels *= grmul
        out_channels = int(int(out_channels + 1) / 2) * 2
        in_channels = 0
        for i in link:
          ch,_,_ = self.get_link(i, base_ch, growth_rate, grmul)
          in_channels += ch
        return out_channels, in_channels, link

    def get_out_ch(self):
        return self.out_channels

    def __init__(self, in_channels, growth_rate, grmul, n_layers, dwconv=False):
        super().__init__()
        self.links = []
        conv_layers_ = []
        bnrelu_layers_ = []
        self.layer_bias = []
        self.out_channels = 0
        self.out_partition = collections.defaultdict(list)

        for i in range(n_layers):
          outch, inch, link = self.get_link(i+1, in_channels, growth_rate, grmul)
          self.links.append(link)
          for j in link:
            self.out_partition[j].append(outch)

        cur_ch = in_channels
        for i in range(n_layers):
          accum_out_ch = sum( self.out_partition[i] )
          real_out_ch = self.out_partition[i][0]
          #print( self.links[i],  self.out_partition[i], accum_out_ch)
          conv_layers_.append( nn.Conv2d(cur_ch, accum_out_ch, kernel_size=3, stride=1, padding=1, bias=True) )
          bnrelu_layers_.append( BRLayer(real_out_ch) )
          cur_ch = real_out_ch
          if (i % 2 == 0) or (i == n_layers - 1):
            self.out_channels += real_out_ch
        #print("Blk out =",self.out_channels)

        self.conv_layers = nn.ModuleList(conv_layers_)
        self.bnrelu_layers = nn.ModuleList(bnrelu_layers_)
    
    def transform(self, blk, trt=False):
        # Transform weight matrix from a pretrained HarDBlock v1
        in_ch = blk.layers[0][0].weight.shape[1]
        for i in range(len(self.conv_layers)):
            link = self.links[i].copy()
            link_ch = [blk.layers[k-1][0].weight.shape[0] if k > 0 else 
                       blk.layers[0  ][0].weight.shape[1] for k in link]
            part = self.out_partition[i]
            w_src = blk.layers[i][0].weight
            b_src = blk.layers[i][0].bias
            
            
            self.conv_layers[i].weight[0:part[0], :, :,:] = w_src[:, 0:in_ch, :,:]
            self.layer_bias.append(b_src)
            
            if b_src is not None:
                if trt:
                    self.conv_layers[i].bias[1:part[0]] = b_src[1:]
                    self.conv_layers[i].bias[0] = b_src[0]
                    self.conv_layers[i].bias[part[0]:] = 0
                    self.layer_bias[i] = None
                else:
                    #for pytorch, add bias with standalone tensor is more efficient than within conv.bias
                    #this is because the amount of non-zero bias is small, 
                    #but if we use conv.bias, the number of bias will be much larger
                    self.conv_layers[i].bias = None
            else:
                self.conv_layers[i].bias = None 

            in_ch = part[0]
            link_ch.reverse()
            link.reverse()
            if len(link) > 1:
                for j in range(1, len(link) ):
                    ly  = link[j]
                    part_id  = self.out_partition[ly].index(part[0])
                    chos = sum( self.out_partition[ly][0:part_id] )
                    choe = chos + part[0]
                    chis = sum( link_ch[0:j] )
                    chie = chis + link_ch[j]
                    self.conv_layers[ly].weight[chos:choe, :,:,:] = w_src[:, chis:chie,:,:]
            
            #update BatchNorm or remove it if there is no BatchNorm in the v1 block
            self.bnrelu_layers[i] = None
            if isinstance(blk.layers[i][1], nn.BatchNorm2d):
                self.bnrelu_layers[i] = nn.Sequential(
                         blk.layers[i][1],
                         blk.layers[i][2])
            else:
                self.bnrelu_layers[i] = blk.layers[i][1]
                    

    def forward(self, x):
        layers_ = []
        outs_ = []
        xin = x
        for i in range(len(self.conv_layers)):
            link = self.links[i]
            part = self.out_partition[i]

            xout = self.conv_layers[i](xin)
            layers_.append(xout)

            xin = xout[:,0:part[0],:,:] if len(part) > 1 else xout
            if self.layer_bias[i] is not None:
                xin += self.layer_bias[i].view(1,-1,1,1)

            if len(link) > 1:
                for j in range( len(link) - 1 ):
                    ly  = link[j]
                    part_id  = self.out_partition[ly].index(part[0])
                    chs = sum( self.out_partition[ly][0:part_id] )
                    che = chs + part[0]                    
                    
                    xin += layers_[ly][:,chs:che,:,:]
                    
            xin = self.bnrelu_layers[i](xin)

            if i%2 == 0 or i == len(self.conv_layers)-1:
              outs_.append(xin)

        out = torch.cat(outs_, 1)
        return out


class HarDBlock(nn.Module):
    def get_link(self, layer, base_ch, growth_rate, grmul):
        if layer == 0:
          return base_ch, 0, []
        out_channels = growth_rate
        link = []
        for i in range(10):
          dv = 2 ** i
          if layer % dv == 0:
            k = layer - dv
            link.append(k)
            if i > 0:
                out_channels *= grmul
        out_channels = int(int(out_channels + 1) / 2) * 2
        in_channels = 0
        for i in link:
          ch,_,_ = self.get_link(i, base_ch, growth_rate, grmul)
          in_channels += ch
        return out_channels, in_channels, link

    def get_out_ch(self):
        return self.out_channels
 
    def __init__(self, in_channels, growth_rate, grmul, n_layers, keepBase=False, residual_out=False):
        super().__init__()
        self.in_channels = in_channels
        self.growth_rate = growth_rate
        self.grmul = grmul
        self.n_layers = n_layers
        self.keepBase = keepBase
        self.links = []
        layers_ = []
        self.out_channels = 0 # if upsample else in_channels
        for i in range(n_layers):
          outch, inch, link = self.get_link(i+1, in_channels, growth_rate, grmul)
          self.links.append(link)
          use_relu = residual_out
          layers_.append(ConvLayer(inch, outch))
          if (i % 2 == 0) or (i == n_layers - 1):
            self.out_channels += outch
        #print("Blk out =",self.out_channels)
        self.layers = nn.ModuleList(layers_)


    def forward(self, x):
        layers_ = [x]
        for layer in range(len(self.layers)):
            link = self.links[layer]
            tin = []
            for i in link:
                tin.append(layers_[i])
            if len(tin) > 1:
                x = torch.cat(tin, 1)
            else:
                x = tin[0]
            out = self.layers[layer](x)
            layers_.append(out)
        t = len(layers_)
        out_ = []
        for i in range(t):
          if (i == 0 and self.keepBase) or \
             (i == t-1) or (i%2 == 1):
              out_.append(layers_[i])
        out = torch.cat(out_, 1)
        return out



class TransitionUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        #print("upsample",in_channels, out_channels)

    def forward(self, x, skip, concat=True):
        out = torch.nn.functional.interpolate(
                x,
                size=(skip.size(2), skip.size(3)),
                mode="bilinear",
                align_corners=True,
                            )
        if concat:                            
          out = torch.cat([out, skip], 1)
          
        return out

class hardnet(nn.Module):
    def __init__(self, n_classes=19):
        super(hardnet, self).__init__()

        first_ch  = [16,24,32,48]
        ch_list = [  64, 96, 160, 224, 320]
        grmul = 1.7
        gr       = [  10,16,18,24,32]
        n_layers = [   4, 4, 8, 8, 8]

        blks = len(n_layers) 
        self.shortcut_layers = []

        self.base = nn.ModuleList([])
        self.base.append (
             ConvLayer(in_channels=3, out_channels=first_ch[0], kernel=3,
                       stride=2) )
        self.base.append ( ConvLayer(first_ch[0], first_ch[1],  kernel=3) )
        self.base.append ( ConvLayer(first_ch[1], first_ch[2],  kernel=3, stride=2) )
        self.base.append ( ConvLayer(first_ch[2], first_ch[3],  kernel=3) )

        skip_connection_channel_counts = []
        ch = first_ch[3]
        for i in range(blks):
            blk = HarDBlock(ch, gr[i], grmul, n_layers[i])
            ch = blk.get_out_ch()
            skip_connection_channel_counts.append(ch)
            self.base.append ( blk )
            if i < blks-1:
              self.shortcut_layers.append(len(self.base)-1)

            self.base.append ( ConvLayer(ch, ch_list[i], kernel=1) )
            ch = ch_list[i]
            
            if i < blks-1:            
              self.base.append ( nn.AvgPool2d(kernel_size=2, stride=2) )


        cur_channels_count = ch
        prev_block_channels = ch
        n_blocks = blks-1
        self.n_blocks =  n_blocks

        #######################
        #   Upsampling path   #
        #######################

        self.transUpBlocks = nn.ModuleList([])
        self.denseBlocksUp = nn.ModuleList([])
        self.conv1x1_up    = nn.ModuleList([])
        
        for i in range(n_blocks-1,-1,-1):
            self.transUpBlocks.append(TransitionUp(prev_block_channels, prev_block_channels))
            cur_channels_count = prev_block_channels + skip_connection_channel_counts[i]
            self.conv1x1_up.append(ConvLayer(cur_channels_count, cur_channels_count//2, kernel=1))
            cur_channels_count = cur_channels_count//2

            blk = HarDBlock(cur_channels_count, gr[i], grmul, n_layers[i])
            
            self.denseBlocksUp.append(blk)
            prev_block_channels = blk.get_out_ch()
            cur_channels_count = prev_block_channels


        self.finalConv = nn.Conv2d(in_channels=cur_channels_count,
               out_channels=n_classes, kernel_size=1, stride=1,
               padding=0, bias=True)
    
    def v2_transform(self, trt=False):        
        for i in range( len(self.base)):
            if isinstance(self.base[i], HarDBlock):
                blk = self.base[i]
                self.base[i] = HarDBlock_v2(blk.in_channels, blk.growth_rate, blk.grmul, blk.n_layers)
                self.base[i].transform(blk, trt)

        for i in range(self.n_blocks):
            blk = self.denseBlocksUp[i]
            self.denseBlocksUp[i] = HarDBlock_v2(blk.in_channels, blk.growth_rate, blk.grmul, blk.n_layers)
            self.denseBlocksUp[i].transform(blk, trt)

    def forward(self, x):
        
        skip_connections = []
        size_in = x.size()
        
        
        for i in range(len(self.base)):
            x = self.base[i](x)
            if i in self.shortcut_layers:
                skip_connections.append(x)
        out = x
        
        for i in range(self.n_blocks):
            skip = skip_connections.pop()
            out = self.transUpBlocks[i](out, skip, True)
            out = self.conv1x1_up[i](out)
            out = self.denseBlocksUp[i](out)
        
        out = self.finalConv(out)
        
        out = torch.nn.functional.interpolate(
                            out,
                            size=(size_in[2], size_in[3]),
                            mode="bilinear",
                            align_corners=True)
        return out

In [ ]:
import torch.nn as nn
import torch.nn.functional as Fun
import torch 

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=0, size_average=True, ignore_index=255):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.size_average = size_average

    def forward(self, inputs, targets):
        ce_loss = Fun.cross_entropy(
            inputs, targets, reduction='none', ignore_index=self.ignore_index)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        if self.size_average:
            return focal_loss.mean()
        else:
            return focal_loss.sum()


import numpy as np
from sklearn.metrics import confusion_matrix

class _StreamMetrics(object):
    def __init__(self):
        """ Overridden by subclasses """
        raise NotImplementedError()

    def update(self, gt, pred):
        """ Overridden by subclasses """
        raise NotImplementedError()

    def get_results(self):
        """ Overridden by subclasses """
        raise NotImplementedError()

    def to_str(self, metrics):
        """ Overridden by subclasses """
        raise NotImplementedError()

    def reset(self):
        """ Overridden by subclasses """
        raise NotImplementedError()      

class StreamSegMetrics(_StreamMetrics):
    """
    Stream Metrics for Semantic Segmentation Task
    """
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.confusion_matrix = np.zeros((n_classes, n_classes))

    def update(self, label_trues, label_preds):
        #boolarr=label_trues==255
        #label_preds[boolarr]=255
        for lt, lp in zip(label_trues, label_preds):
            self.confusion_matrix += self._fast_hist( lt.flatten(), lp.flatten() )
    
    @staticmethod
    def to_str(results):
        string = "\n"
        for k, v in results.items():
            if k!="Class IoU":
                string += "%s: %f\n"%(k, v)
        
        #string+='Class IoU:\n'
        #for k, v in results['Class IoU'].items():
        #    string += "\tclass %d: %f\n"%(k, v)
        return string

    def _fast_hist(self, label_true, label_pred):
        mask = (label_true >= 0) & (label_true < self.n_classes)
        hist = np.bincount(
            self.n_classes * label_true[mask].astype(int) + label_pred[mask],
            minlength=self.n_classes ** 2,
        ).reshape(self.n_classes, self.n_classes)
        return hist

    def get_results(self):
        """Returns accuracy score evaluation result.
            - overall accuracy
            - mean accuracy
            - mean IU
            - fwavacc
        """
        hist = self.confusion_matrix
        acc = np.diag(hist).sum() / hist.sum()
        acc_cls = np.diag(hist) / hist.sum(axis=1)
        acc_cls = np.nanmean(acc_cls)
        iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
        mean_iu = np.nanmean(iu)
        freq = hist.sum(axis=1) / hist.sum()
        fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
        cls_iu = dict(zip(range(self.n_classes), iu))
#         cls_ac = dict(zip(range(self.n_classes),np.diag(hist)/(hist.sum(axis=1))))

        return {
                "Overall Acc": acc,
                "Mean Acc": acc_cls,
                "FreqW Acc": fwavacc,
                "Mean IoU": mean_iu,
                #"Class Acc": cls_ac,
                "Class IoU": cls_iu,
            }
        
    def reset(self):
        self.confusion_matrix = np.zeros((self.n_classes, self.n_classes))

class AverageMeter(object):
    """Computes average values"""
    def __init__(self):
        self.book = dict()

    def reset_all(self):
        self.book.clear()
    
    def reset(self, id):
        item = self.book.get(id, None)
        if item is not None:
            item[0] = 0
            item[1] = 0

    def update(self, id, val):
        record = self.book.get(id, None)
        if record is None:
            self.book[id] = [val, 1]
        else:
            record[0]+=val
            record[1]+=1

    def get_results(self, id):
        record = self.book.get(id, None)
        assert record is not None
        return record[0] / record[1]


In [ ]:
# import numpy as np
# a=np.array([1,3,4,14,1,2,42,2])
# boolarr=a==1
# b=np.array([5,3,4,14,10,2,42,100])
# b[boolarr]=255
# print(b)

In [ ]:
_pil_interpolation_to_str = {
    Image.NEAREST: 'PIL.Image.NEAREST',
    Image.BILINEAR: 'PIL.Image.BILINEAR',
    Image.BICUBIC: 'PIL.Image.BICUBIC',
    Image.LANCZOS: 'PIL.Image.LANCZOS',
    Image.HAMMING: 'PIL.Image.HAMMING',
    Image.BOX: 'PIL.Image.BOX',
}
class ExtResize(object):
    """Resize the input PIL Image to the given size.
    Args:
        size (sequence or int): Desired output size. If size is a sequence like
            (h, w), output size will be matched to this. If size is an int,
            smaller edge of the image will be matched to this number.
            i.e, if height > width, then image will be rescaled to
            (size * height / width, size)
        interpolation (int, optional): Desired interpolation. Default is
            ``PIL.Image.BILINEAR``
    """

    def __init__(self, size, interpolation=Image.BILINEAR):
        #assert isinstance(size, int) or (isinstance(size, collections.Iterable) and len(size) == 2)
        self.size = size
        self.interpolation = interpolation

    def __call__(self, img, lbl):
        """
        Args:
            img (PIL Image): Image to be scaled.
        Returns:
            PIL Image: Rescaled image.
        """
        return F.resize(img, self.size, self.interpolation), F.resize(lbl, self.size, Image.NEAREST)

    def __repr__(self):
        interpolate_str = _pil_interpolation_to_str[self.interpolation]
        return self.__class__.__name__ + '(size={0}, interpolation={1})'.format(self.size, interpolate_str) 


In [ ]:
os.chdir("/kaggle/input/")

In [ ]:
!ls

In [ ]:
os.chdir("./script/")

In [ ]:
!python utils.py

In [ ]:
from utils import *

In [ ]:
!ls
os.chdir("..")
os.chdir('../working')

In [ ]:
import json
import os
from collections import namedtuple

import torch
import torch.utils.data as data
from PIL import Image
import numpy as np


class CustomData(data.Dataset):

    # Based on https://github.com/mcordts/cityscapesScripts
    colorMap={
        "backgroud": (225,229 , 204),
        'wall':(152, 152, 79),
        'building':(70, 70, 70),
        'sky':(70, 130, 180),
        'sidewalk':(244, 35, 232),
        'field/grass':(152, 251, 152),
        'vegitation':(107, 142, 35),
        'person': (220, 20, 60),
        'mountain':(139, 218, 51),
        'stairs':(202, 251, 254),
        'bench':(108, 246, 107),
        'pole':(153, 153, 153),
        'car':(41, 34, 177),
        'bike':(111, 34, 177),
        'animal':(211, 205, 33),
        'ground':(147, 147, 136),
        'fence':(241, 170, 17),
        'water':(29, 231, 229),
        'road':(35, 18, 16),
        'sign_board':(113, 97, 41),
        'floor':(73, 23, 77),
        'traffic_light':(225, 175, 57),
        'ceeling':(51, 0, 0),
        'unlabelled':(0,0,0),
        
    }
     
    # Based on https://github.com/mcordts/cityscapesScripts
    CustomDataSet = namedtuple('CustomDataSet', ['name', 'id', 'train_id', 'category', 'category_id',
                                                     'has_instances', 'ignore', 'color'])
    classes = [
            CustomDataSet('backgroud',       0, 0, 'obstacle', 0, False, True, colorMap['backgroud']),
            CustomDataSet('wall',            1, 1, 'solid', 0, False, True, colorMap['wall']),
            CustomDataSet('building',        2, 2, 'solid', 0, False, True, colorMap['building']),
            CustomDataSet('sky',             3, 3, 'backgroud', 0, False, True, colorMap['sky']),
            CustomDataSet('sidewalk',        4, 4, 'nature', 0, False, True, colorMap['sidewalk']),
            CustomDataSet('field/grass',     5, 5, 'nature', 0, False, True, colorMap['field/grass']),
            CustomDataSet('vegitation',      6, 6, 'nature', 0, False, True, colorMap['vegitation']),
            CustomDataSet('person',          7, 7, 'human', 0, False, True, colorMap['person']),
            CustomDataSet('mountain',        8, 8, 'nature', 0, False, True, colorMap['mountain']),
            CustomDataSet('stairs',          9, 255, 'solid', 0, False, False, colorMap['stairs']),
            CustomDataSet('bench',           10, 0, 'obstacle', 0, False, False, colorMap['bench']),
            CustomDataSet('pole',            11, 0, 'obstacle', 0, False, False, colorMap['pole']),
            CustomDataSet('car',             12, 9, 'vahicle', 0, False, True, colorMap['car']),
            CustomDataSet('bike',            13, 10, 'vahicle', 0, False, True, colorMap['bike']),
            CustomDataSet('animal',          14, 11, 'animal', 0, False, True, colorMap['animal']),
            CustomDataSet('ground',          15, 12, 'land', 0, False, True, colorMap['ground']),
            CustomDataSet('fence',           16, 13, 'solid', 0, False, True, colorMap['fence']),
            CustomDataSet('water',           17, 14, 'land', 0, False, True, colorMap['water']),
            CustomDataSet('road',            18, 15, 'land', 0, False, True, colorMap['road']),
            CustomDataSet('sign_board',      19, 0, 'obstacle', 0, False, False, colorMap['sign_board']),
            CustomDataSet('floor',           20, 4, 'land', 0, False, False, colorMap['floor']),
            CustomDataSet('traffic_light',   21, 0,'obstacle', 0, False, False, colorMap['traffic_light']),
            CustomDataSet('ceeling',         22, 16, 'ceeling', 0, False, True, colorMap['ceeling']),
            CustomDataSet('unlabelled',      23, 255, 'void', 0, False, False, colorMap['unlabelled']),
            
    ]
    
#     train_id_to_color = [c.color for c in classes if (c.train_id != -1 and c.train_id != 255)]
#     train_id_to_color.append([0, 0, 0])
#     train_id_to_color = np.array(train_id_to_color)
    
#     train_id_to_label= [c.name for c in classes if (c.train_id != -1 and c.train_id != 255)]
#     train_id_to_label.append("unlabeled")
#     train_id_to_label = np.array(train_id_to_label)

    train_id_to_color = [c.color for c in classes if (c.ignore)]
    train_id_to_color.append([0, 0, 0])
    train_id_to_color = np.array(train_id_to_color)

    train_id_to_label= [c.name for c in classes if (c.ignore)]
    train_id_to_label.append("unlabeled")
    train_id_to_label = np.array(train_id_to_label)
    
    
    id_to_train_id = np.array([c.train_id for c in classes])
    
    def __init__(self, root_image, root_target, split='train', mode='fine', target_type='semantic', transform=None):        
        
        self.root_image = [os.path.expanduser(i) for i in root_image ]
        self.root_target = [os.path.expanduser(i) for i in root_target]
        
        self.images = []
        self.targets = []
        self.transform = transform
        self.split = split
        
        for i in range(len(self.root_image)):
            self.root_image[i]=os.path.join(self.root_image[i],split)
            self.root_target[i]=os.path.join(self.root_target[i],split)
            print(self.root_image[i])
            print(self.root_target[i])

            lst=os.listdir(self.root_target[i])
            
            for img in lst:
                if img[0]=="C":
                    self.images.append(os.path.join(root_image[i],img[:-4]+'.jpg'))
                else:
                    self.images.append(os.path.join(self.root_image[i],img[:-4]+'.jpg'))
                self.targets.append(os.path.join(self.root_target[i],img))
        print(len(self.images))
        print(len(self.targets))


    @classmethod
    def encode_target(cls, target):
        return cls.id_to_train_id[np.array(target)]

    @classmethod
    def decode_target(cls, target):
        target[target == 255] = 0
        #target = target.astype('uint8') + 1
        return cls.train_id_to_color[target]

    def __getitem__(self, index):
        image = Image.open(self.images[index]).convert('RGB')
        target = Image.open(self.targets[index])
        
       
        if self.transform:
            image, target = self.transform(image, target)
        target = self.encode_target(target)
        #print(image.shape)
        #print(target.shape)
        
        return image, target

    def __len__(self):
        return len(self.images)

In [ ]:
len(CustomData.train_id_to_color)

In [ ]:
CustomData.train_id_to_label

In [ ]:
CustomData.id_to_train_id

In [ ]:
from torchvision.transforms.functional import normalize
import torch.nn as nn
import numpy as np
import os 

def denormalize(tensor, mean, std):
    mean = np.array(mean)
    std = np.array(std)

    _mean = -mean/std
    _std = 1/std
    return normalize(tensor, _mean, _std)

class Denormalize(object):
    def __init__(self, mean, std):
        mean = np.array(mean)
        std = np.array(std)
        self._mean = -mean/std
        self._std = 1/std

    def __call__(self, tensor):
        if isinstance(tensor, np.ndarray):
            return (tensor - self._mean.reshape(-1,1,1)) / self._std.reshape(-1,1,1)
        return normalize(tensor, self._mean, self._std)

def set_bn_momentum(model, momentum=0.1):
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.momentum = momentum

def fix_bn(model):
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()

def mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [ ]:
from matplotlib import gridspec

def create_label_colormap():
    colormap = CustomData.train_id_to_color
    return colormap


def label_to_color_image(label):
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]


def vis_segmentation(image, seg_map):
    """Visualizes input image, segmentation map and overlay view."""
    plt.figure(figsize=(20, 4))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image =  CustomData.decode_target(output_predictions.cpu()).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
    print("Uniques Labels Found",unique_labels)
    ax = plt.subplot(grid_spec[3])
    
    
    plt.imshow(FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    plt.show()


LABEL_NAMES = CustomData.train_id_to_label

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [ ]:
class opts:
    dataset="cityscapes"

    data_root="../input/coco-dataset/Coco Stuff Dataset/imageLists"
  
    imageFolder= [
                  '../input/adek20-screen-parsing/ADEChallengeData2016/images',
                  '../input/adek20-screen-parsing/ADEChallengeData2016/images',
                  '../input/coco-dataset/Coco Stuff Dataset/images',
                  '../input/custom-sidewalk/custom_sidewalk_updated/customdataset',
                 ]
    
    targetFolder=[
        '../input/adk-coco-filter/ADK_COCO_Filter_Anotation_Updated/adk',
                  '../input/adk-coco-filter/ADK_COCO_Filter_Anotation_Updated/adk_floor_filter',
                  '../input/adk-coco-filter/ADK_COCO_Filter_Anotation_Updated/coco',
                  '../input/custom-sidewalk/custom_sidewalk_updated/customannotations',
                 ]
    lr=0.00001
    weight_decay=0.0001
    lr_policy= 'poly'  #learning rate scheduler policy
    step_size=10000
    saved_path="../input/fdnet-results-16-classes/Files/"
    pretrained=True
    continue_training=True   #train previous model
    batch_size=16
    val_batch_size=16
    num_classes=17
    loss_function="cross_entropy" #"cross_entropy

<!-- ### import numpy as np
# from sklearn.metrics import confusion_matrix

# class _StreamMetrics(object):
#     def __init__(self):
#         """ Overridden by subclasses """
#         raise NotImplementedError()

#     def update(self, gt, pred):
#         """ Overridden by subclasses """
#         raise NotImplementedError()

#     def get_results(self):
#         """ Overridden by subclasses """
#         raise NotImplementedError()

#     def to_str(self, metrics):
#         """ Overridden by subclasses """
#         raise NotImplementedError()

#     def reset(self):
#         """ Overridden by subclasses """
#         raise NotImplementedError()      

# class StreamSegMetrics(_StreamMetrics):
#     """
#     Stream Metrics for Semantic Segmentation Task
#     """
#     def __init__(self, n_classes):
#         self.n_classes = n_classes
#         self.confusion_matrix = np.zeros((n_classes, n_classes))

#     def update(self, label_trues, label_preds):
#         for lt, lp in zip(label_trues, label_preds):
#             self.confusion_matrix += self._fast_hist( lt.flatten(), lp.flatten() )
    
#     @staticmethod
#     def to_str(results):
#         string = "\n"
#         for k, v in results.items():
#             if k!="Class IoU":
#                 string += "%s: %f\n"%(k, v)
        
#         #string+='Class IoU:\n'
#         #for k, v in results['Class IoU'].items():
#         #    string += "\tclass %d: %f\n"%(k, v)
#         return string

#     def _fast_hist(self, label_true, label_pred):
#         mask = (label_true >= 0) & (label_true < self.n_classes)
#         hist = np.bincount(
#             self.n_classes * label_true[mask].astype(int) + label_pred[mask],
#             minlength=self.n_classes ** 2,
#         ).reshape(self.n_classes, self.n_classes)
#         return hist

#     def get_results(self):
#         """Returns accuracy score evaluation result.
#             - overall accuracy
#             - mean accuracy
#             - mean IU
#             - fwavacc
#         """
#         hist = self.confusion_matrix
#         acc = np.diag(hist).sum() / hist.sum()
#         acc_cls = np.diag(hist) / hist.sum(axis=1)
#         acc_cls = np.nanmean(acc_cls)
#         iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
#         mean_iu = np.nanmean(iu)
#         freq = hist.sum(axis=1) / hist.sum()
#         fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
#         cls_iu = dict(zip(range(self.n_classes), iu))

#         return {
#                 "Overall Acc": acc,
#                 "Mean Acc": acc_cls,
#                 "FreqW Acc": fwavacc,
#                 "Mean IoU": mean_iu,
#                 "Class IoU": cls_iu,
#             }
        
#     def reset(self):
#         self.confusion_matrix = np.zeros((self.n_classes, self.n_classes))

# class AverageMeter(object):
#     """Computes average values"""
#     def __init__(self):
#         self.book = dict()

#     def reset_all(self):
#         self.book.clear()
    
#     def reset(self, id):
#         item = self.book.get(id, None)
#         if item is not None:
#             item[0] = 0
#             item[1] = 0

#     def update(self, id, val):
#         record = self.book.get(id, None)
#         if record is None:
#             self.book[id] = [val, 1]
#         else:
#             record[0]+=val
#             record[1]+=1

#     def get_results(self, id):
#         record = self.book.get(id, None)
#         assert record is not None
#         return record[0] / record[1] -->

In [ ]:

def get_dataset(opts):
    """ Dataset And Augmentation
    """
    if opts.dataset == 'cityscapes':
        train_transform = ExtCompose([
            #et.ExtResize( 512 ),
            ExtResize((640,480)),
            #ExtRandomCrop(size=(opts.crop_size, opts.crop_size),pad_if_needed=True),
            ExtColorJitter( brightness=0.5, contrast=0.5, saturation=0.5 ),
            ExtRandomHorizontalFlip(),
            ExtToTensor(),
            ExtNormalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
        ])
        val_transform = ExtCompose([
            #et.ExtResize( 512 ),
            ExtResize((640,480)),
            ExtColorJitter( brightness=0.5, contrast=0.5, saturation=0.5 ),
            #ExtRandomCrop(size=(opts.crop_size, opts.crop_size)),
            ExtToTensor(),
            ExtNormalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
        ])

        train_dst = CustomData(root_image=opts.imageFolder,root_target=opts.targetFolder,
                               split='training', transform=train_transform)
        val_dst = CustomData(root_image=opts.imageFolder,root_target=opts.targetFolder,
                             split='validation', transform=val_transform)
    return train_dst, val_dst

In [ ]:
train_dst, val_dst = get_dataset(opts)
train_loader = data.DataLoader(train_dst, batch_size=opts.batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = data.DataLoader(val_dst, batch_size=opts.val_batch_size, shuffle=True, num_workers=4)
print("Dataset: %s, Train set: %d, Val set: %d" % (opts.dataset, len(train_dst), len(val_dst)))

In [ ]:
dataset_iter = iter(train_loader)
img,lal=dataset_iter.next()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: %s" % device)

In [ ]:
model=hardnet(19)


In [ ]:
def dfs_freeze(model):
    for name, child in model.named_children():
        for param in child.parameters():
            param.requires_grad = False
        dfs_freeze(child)

In [ ]:
def save_ckpt(path):
    """ save current model
    """
    torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "scheduler_state": scheduler.state_dict(),
            "best_score": best_score,
    }, path)
    print("Model saved as %s" % path)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=opts.lr, momentum=0.9, weight_decay=opts.weight_decay)
metrics = StreamSegMetrics(opts.num_classes)
creterion=""
if opts.loss_function=="focal":
    criterion = FocalLoss(ignore_index=255, size_average=True)
else:
    criterion = nn.CrossEntropyLoss(ignore_index=255, reduction='mean')
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=opts.step_size, gamma=0.1)

In [ ]:

#criterion.to(device)

cur_epoch=0
best_score=0
Loss=[]
Score=[]

if opts.pretrained:
    model.finalConv= nn.Conv2d(48, opts.num_classes, 1, 1, bias=True) 
    ckpt=opts.saved_path+"last_model.pth"
    checkpoint = torch.load(ckpt, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint["model_state"])
    #model = nn.DataParallel(model)
    model.to(device)
    with open(opts.saved_path+"Loss.txt","rb") as File:
        Loss = pickle.load(File)
    with open(opts.saved_path+"Score.txt","rb") as File:
        Score = pickle.load(File)
        
    if opts.continue_training:
        optimizer.load_state_dict(checkpoint["optimizer_state"])
        scheduler.load_state_dict(checkpoint["scheduler_state"])
        cur_epoch = checkpoint["epoch"]
        best_score = checkpoint['best_score']
        print("Training state restored from %s" % opts.saved_path)
    print("Model restored from %s" % opts.saved_path)
    del checkpoint  # free memory
else:
    model = torch.nn.DataParallel(model)
    checkpoint=torch.load('../input/cityscape-mobilenet/hardnet70_cityscapes_model_2.pkl',map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint["model_state"])
    model=model.module
    model.finalConv= nn.Conv2d(48, opts.num_classes, 1, 1, bias=True) 
    #model = nn.DataParallel(model)
    model.to(device)

In [ ]:
print(len(Loss))

In [ ]:
print(cur_epoch)

In [ ]:

def validate1(opts, model,loader,metrics,device):
    
    metrics.reset()
    #validation_loss = []
    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images = images.to(device, dtype=torch.float32)
            labels = labels.to(device, dtype=torch.long)

            outputs = model(images)
            preds = outputs.detach().max(dim=1)[1].cpu().numpy()
            targets=labels.cpu().numpy()
            #loss = criterion(outputs, labels)
            
            
            metrics.update(targets, preds)
            #np_loss=loss.detach().cpu().numpy() 
            #validation_loss.append(np_loss)
    
    #loss=np.mean(validation_loss)
    #print(validation_loss)
    return metrics.get_results()

In [ ]:
mkdir("./Files")

In [ ]:
interval_loss = 0
epochs=cur_epoch+58
max_score=10000000000
model_curve=[]



best_score = 0.0
cur_itrs=0
start = perf_counter()
for epoch in tqdm(range(cur_epoch+1,epochs), desc="Epochs"):
    model.train()
    running_loss = []
    for step, (images, labels) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        cur_itrs += 1
        #print(images.shape)
        #print(labels.shape)
        
        images = images.to(device, dtype=torch.float32)
        labels = labels.to(device, dtype=torch.long)
        optimizer.zero_grad()

        outputs = model(images)
        #print(outputs.shape)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #end = perf_counter()
        
        running_loss.append(loss.item())
        interval_loss += loss.item()

    model.eval()
    val_score = validate1(opts=opts, model=model, loader=val_loader, metrics=metrics,device=device)
    print("Training Loss",np.mean(running_loss),"Validation Loss")
    print(metrics.to_str(val_score))

    #if val_score['Mean IoU'] > best_score:  # save best model
    best_score = val_score['Mean IoU']
    save_ckpt('./Files/last_model.pth')
#     else:
#         print("Stopping Condition Occoured")
    Loss.append(np.mean(running_loss))
    Score.append(val_score)
    
    with open("./Files/Loss.txt","wb") as File:
        pickle.dump(Loss,File)
    with open("./Files/Score.txt","wb") as File:
        pickle.dump(Score,File)

    scheduler.step()
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(running_loss)))
end = perf_counter()
print("Time",end-start)

In [ ]:
print(Score[-1])

In [ ]:
def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

In [ ]:
zipf = zipfile.ZipFile("./Files.zip", 'w', zipfile.ZIP_DEFLATED)
zipdir('./Files', zipf)
zipf.close()

In [ ]:
from PIL import Image
from torchvision import transforms

from PIL import Image
import requests

In [ ]:
#start1 = perf_counter()
input_image = Image.open("../input/my-dataset/images/6.jpg")
#url="https://images.squarespace-cdn.com/content/v1/5c9b3ac2e5f7d1493cede23f/1554710179935-T2AY655VMKGCZ6V5D0AE/ke17ZwdGBToddI8pDm48kBqZKGEd5IotpaYmRYDAsV97gQa3H78H3Y0txjaiv_0fDoOvxcdMmMKkDsyUqMSsMWxHk725yiiHCCLfrh8O1z5QPOohDIaIeljMHgDF5CVlOqpeNLcJ80NK65_fV7S1UYXHvO3AlG5qAFDx5BBWbEnTecsx1VxL9XkiyXErq2cftN_KfIuurh-w-x7bjNuMJA/london-citizen-m-stairs-15.jpg"
#input_image=Image.open(requests.get(url, stream=True).raw)
preprocess_input= transforms.Compose([
    transforms.Resize((480,640)),
])
preprocess = transforms.Compose([
    transforms.Resize((480,640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.ColorJitter( brightness=0.5, contrast=0.5, saturation=0.5 ),
])
input_image=preprocess_input(input_image)
input_tensor= preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

model.eval()
with torch.no_grad():
    start = perf_counter()
    output = model(input_batch)[0]
    end = perf_counter()
    print(end-start)

output_predictions = output.argmax(0)
pred = CustomData.decode_target(output_predictions.cpu()).astype(np.uint8)

In [ ]:
vis_segmentation(input_image, output_predictions.cpu())

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((480,640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
def run_model(input_image):
    input_tensor= preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) 
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')
    model.eval()
    with torch.no_grad():
        output = model(input_batch)[0]
    output_predictions = output.argmax(0)
    return output_predictions

In [ ]:
import cv2
import urllib
import IPython
from io import BytesIO
import matplotlib.pyplot as plt
import io
import pickle

np_buff=[]

def vis_segmentation_stream(image, seg_map, index):
    
    """Visualizes segmentation overlay view and stream it with IPython display."""
#     fig=plt.figure(figsize=(12, 7))
    seg_image = CustomData.decode_target(seg_map.cpu()).astype(np.uint8)
    seg_image=Image.fromarray(seg_image.astype('uint8'), 'RGB')

    background = image.convert("RGBA")
    overlay = seg_image.convert("RGBA")

    new_img = Image.blend(background, overlay, 0.7)
    #new_img.save("new.png","PNG")
    img=np.array(new_img)
    #print(img.shape)
    np_buff.append(img)


def run_visualization_video(frame, index):
    """Inferences DeepLab model on a video file and stream the visualization."""
    #frame=preprocess_input(frame)
    original_im = Image.fromarray(frame[..., ::-1])
    original_im=preprocess_input(original_im)
    seg_map = run_model(original_im)
    vis_segmentation_stream(original_im, seg_map, index)


SAMPLE_VIDEO = '../input/video-test/xyz.mp4'
if not os.path.isfile(SAMPLE_VIDEO): 
    print('downloading the sample video...')
    SAMPLE_VIDEO = urllib.request.urlretrieve('https://github.com/lexfridman/mit-deep-learning/raw/master/tutorial_driving_scene_segmentation/mit_driveseg_sample.mp4')[0]
print('running deeplab on the sample video...')

video = cv2.VideoCapture(SAMPLE_VIDEO)
#print(len(video))

# # num_frames = 598  # uncomment to use the full sample video
num_frames = 20000

try:
    start = perf_counter()
    for i in range(num_frames):
        _, frame = video.read()
        if not _: break
        if i%15==0:
            run_visualization_video(frame, i)
    end = perf_counter()
    sec=end-start
    print(sec/60)
except KeyboardInterrupt:
    plt.close()
    print("Stream stopped.")

In [ ]:
video=cv2.VideoWriter("result.mp4",cv2.VideoWriter_fourcc(*'DIVX'), 15,(np_buff[0].shape[1],np_buff[0].shape[0]))
for i in np_buff:
    i=i[:,:,:-1]
    video.write(i[:,:,::-1])
video.release()

In [ ]:
# val_files= os.listdir('../input/floor-separated/Annotation_adk_coco_only/adk/validation')